In [23]:
import pickle
import functools as f
from pymatgen import MPRester
from copy import deepcopy
import pandas as pd
import pymatgen as mg
import pymatgen
from pymatgen.analysis.structure_matcher import StructureMatcher
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core.structure import SiteCollection
from pymatgen.core.bonds import CovalentBond, get_bond_length
from pymatgen.util.coord import get_angle, all_distances, lattice_points_in_supercell
import os
import itertools 

API_KEY = "oq6bIVf0EQMI6dun"
mp = MPRester(API_KEY)
save_path = '/Users/dragonlook/Dropbox (MIT)/MI/codes/MP_data/'

mag_list = pd.read_excel('/Users/dragonlook/Dropbox (MIT)/MI/codes/list/thermal_magnon_list.xlsx')
samples = list(mag_list.Formula.dropna())
mag_elements = list(mag_list.mag_element.dropna())
basic_properties = ["energy", "energy_per_atom", "volume",
                                "formation_energy_per_atom", "nsites",
                                "unit_cell_formula", "pretty_formula",
                                "is_hubbard", "elements", "nelements",
                                "e_above_hull", "hubbards", "is_compatible",
                                "spacegroup", "task_ids", "band_gap", "density",
                                "total_magnetization",
                                "material_id", "oxide_type", "tags", "elasticity"]

In [4]:
def get_data(compound_name):
    basic_properties = ["energy", "energy_per_atom", "volume",
                                "formation_energy_per_atom", "nsites",
                                "unit_cell_formula", "pretty_formula",
                                "is_hubbard", "elements", "nelements",
                                "e_above_hull", "hubbards", "is_compatible",
                                "spacegroup", "task_ids", "band_gap", "density",
                                "total_magnetization",
                                "material_id", "oxide_type", "tags", "elasticity"]

    all_properties = basic_properties

    data = mp.query(criteria=compound_name, properties=all_properties)
    
    return data


In [15]:
# search copper oxide in MP
# criteria = {"elements": {"$all": ["Cu",'O']},"nelements": {"$lte": 3},  "band_gap": {"$exists": True}}
# data = get_data(criteria)
# df = pd.DataFrame(data)
# df.to_csv('/Users/dragonlook/Dropbox (MIT)/MI/codes/list/copper_oxide_three.csv')
# df.to_csv('/Users/dragonlook/Dropbox (MIT)/MI/codes/list/copper_oxide_three.csv', usecols= ['pretty_formula'])


In [47]:
# # pick up copper oxide
# copper_oxide_from_list = []
# for name, ref in zip(samples, mag_elements):
#     if ref == 'Cu':
#         copper_oxide_from_list.append(name)
                                                                                                        

# targets = pd.read_excel('/Users/dragonlook/Dropbox (MIT)/MI/codes/list/thermal_magnon_list.xlsx',usecols=['Formula','mag_element','target'])
# targets = targets.set_index('Formula')
# targets = targets[targets['mag_element'] == 'Cu']
# targets.to_csv('/Users/dragonlook/Dropbox (MIT)/MI/codes/list/copper_oxide_label.csv')

In [3]:
for compound_name in samples:
    compound_name = compound_name.strip()
    
    data = get_data(compound_name)
    df = pd.DataFrame()
    for fts in basic_properties:
        temp = []
        for dat in data:
            temp.append(dat[fts])
        df[fts] = temp

    df.to_csv(save_path + compound_name + '.csv')


In [4]:
sample_inds = len(samples)
for sample_ind in range(sample_inds):
    compound = samples[sample_ind].strip()
    try:
        data_st_s = mp.get_structures(compound)
        new_path = '/Users/dragonlook/Dropbox (MIT)/MI/codes/MP_data/'+str(compound)
        os.makedirs(new_path,exist_ok=True)
        mag_element = pymatgen.core.composition.Element(mag_elements[sample_ind])
        cout = 0
        for data_st in data_st_s: # list of MP structure
            
            # where id mag sites
            temp_cout = -1
            list_of_mag_site = []
            for site in data_st: # list of sites
                temp_cout += 1
                if site.specie == mag_element: #only pick up magnetic components
                    list_of_mag_site.append(temp_cout)
                    
            bond_length_list = []
            bond_angle_list = []
            for mag_site_ind in list_of_mag_site:
                # neighbors of mag sites
                neighors = pymatgen.analysis.local_env.BrunnerNN_real()
                nn_B = neighors.get_nn_info(data_st, mag_site_ind)
                if len(nn_B) == 1 or len(nn_B) == 0:
                    print('CANNOT FIND ENOUGH NEIGHBORS')
                    continue
                    
                else:
                    # generate bond lengths
                    nn_coords = [i['site'].coords for i in nn_B]
                    bond_length_list.append(all_distances([data_st[mag_site_ind].coords],nn_coords)[0])
                    for neighoring_two_sites in itertools.combinations(nn_coords,2):
                        angle = get_angle(neighoring_two_sites[0] - data_st[mag_site_ind].coords, neighoring_two_sites[1] - data_st[mag_site_ind].coords)
                        bond_angle_list.append(angle)
            df_angle = pd.DataFrame()
            df_angle['angle'] = bond_angle_list
            df_length = pd.DataFrame()
            df_length['length'] = bond_length_list
            df_angle.to_csv(new_path + '/' + compound + '_angle_' + str(cout) + '.csv')
            df_length.to_csv(new_path + '/' + compound + '_length_' + str(cout) + '.csv')
            
            cout += 1
            
                
    except Exception as e:
        print(e)
        print(compound)
        print('error: might be no data stored in MP')